In [ ]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
df = pd.read_csv('melb_data.csv')
pd.set_option('display.max_columns', None) 
df

In [ ]:
df.isnull().sum()

In [ ]:
df.info()

In [ ]:
cols = ['Rooms','Bathroom','Bedroom2','Landsize','Lattitude','Longtitude']
X = df[cols]
y = df['Price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = MinMaxScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

model = LinearRegression()

model.fit(X_train_scaled, y_train)

y_pred = model.predict(X_test_scaled)

print(f'MSE: {mean_squared_error(y_test, y_pred):.2f}')
print(f'R^2: {r2_score(y_test, y_pred):.2f}')

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor

cols = ['Rooms','Bathroom','Bedroom2','Landsize','Lattitude','Longtitude']
X = df[cols]
y = df['Price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

param_grid = {
    'max_depth' : [5,10,15,20,None],
    'min_samples_split':[2,10,20,50,100],
    'min_samples_leaf':[1,5,10,20,30]
}

dtr = DecisionTreeRegressor(random_state=1)

grid_search = GridSearchCV(estimator=dtr, param_grid=param_grid, cv=5, scoring='r2', n_jobs=-1)

grid_search.fit(X_train, y_train)

print("Mejores hiperparámetros:", grid_search.best_params_)
print("Mejor R²:", grid_search.best_score_)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

# Distribuciones para la búsqueda aleatoria
param_dist = {
    'max_depth': [5, 10, 15, 20, None],
    'min_samples_split': randint(2, 100),
    'min_samples_leaf': randint(1, 50)
}

# Configurar RandomizedSearch
random_search = RandomizedSearchCV(estimator=dtr, param_distributions=param_dist, n_iter=20, cv=5, scoring='r2', random_state=1, n_jobs=-1)

random_search.fit(X_train, y_train)

print("Mejores hiperparámetros:", random_search.best_params_)
print("Mejor R²:", random_search.best_score_)


Elijo los hiperparámetros obtenidos a partir de 'GridSearchCV' ya que este encontró un límite de profundidad para el árbol, mientras que con 'RandomizedSearchCV' el árbol crece sin límite.

In [ ]:
dtr = DecisionTreeRegressor(max_depth=20, min_samples_split=50, min_samples_leaf=10, random_state=1)
dtr.fit(X_train, y_train)

y_pred = dtr.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'MSE: {mse:.2f}, R^2: {r2:.2f}')

In [ ]:
from sklearn.model_selection import cross_val_score

dtr_best = DecisionTreeRegressor(max_depth=20, min_samples_leaf=12, min_samples_split=30, random_state=1)

cv_scores = cross_val_score(dtr_best, X_train, y_train, cv=5, scoring='r2')

print("R² por fold:", cv_scores)
print("R² promedio:", np.mean(cv_scores))
print("Desviación estándar del R²:", np.std(cv_scores))

In [ ]:
from sklearn.model_selection import cross_val_score

dtr_best = DecisionTreeRegressor(max_depth=20, min_samples_leaf=12, min_samples_split=30, random_state=1)

cv_scores = cross_val_score(dtr_best, X_train, y_train, cv=5, scoring='r2')

print("R² por fold:", cv_scores)
print("R² promedio:", np.mean(cv_scores))
print("Desviación estándar del R²:", np.std(cv_scores))

Se realizó una validación cruzada con 5 folds para evaluar la estabilidad del modelo DecisionTreeRegressor ajustado con los mejores hiperparámetros obtenidos a través de Grid Search. La validación cruzada permite medir el desempeño del modelo de forma más robusta al entrenarlo y evaluarlo en diferentes particiones del conjunto de datos, evitando depender de una única división entre train y test.
La estabilidad del modelo queda reflejada en la baja desviación estándar, indicando que el rendimiento no varía significativamente entre las distintas particiones. Además, el R² promedio (0.6876) es consistente con el R² obtenido sobre el conjunto de test (0.72), lo que sugiere que el modelo tiene un buen equilibrio entre sesgo y varianza, sin señales evidentes de sobreajuste o subajuste.

In [ ]:
#Búsqueda de outliers para las siguientes columnas

cols_out = ['Price','Landsize']

for col in cols_out:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)

    median = df[col].median()
    mean = df[col].mean()

    IQR = Q3-Q1

    llimit = Q1-1.5*Q3
    ulimit = Q3 +1.5*Q1

    df[f'Outlier_{col}'] = (df[col] < llimit) | (df[col] > ulimit)    
    
    print(f"Outliers columna '{col}' {df[f'Outlier_{col}'].value_counts()}")

In [ ]:
df_no_outliers = df[~df['Outlier_Price']]
df_no_outliers = df_no_outliers[~df_no_outliers['Outlier_Landsize']]
df_no_outliers.drop('Outlier', axis=1)
df_no_outliers